<a href="https://colab.research.google.com/github/Orixtosius/Computer-Vision-Exercises/blob/main/Mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import numpy as np
import tensorflow as tf
import logging
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Flatten
from tensorflow.keras.utils import CustomObjectScope
from tensorflow.keras.models import load_model
import cv2
from glob import glob
from tqdm import tqdm

In [4]:
smooth = 1e-15
"""
y is for ground truth
y_hat is for segmentation

"""
def iou(y, y_hat):
  return tf.numpy_function(f, [y, y_hat], dtype = tf.float32)

def f(y, y_hat):
  """
  A ∩ B -> 0/1 so if y and y_hat are both 1, the result is 0.
  Otherwise the result is 0

  A U B = A + B - A ∩ B
  """
  intersection = (y*y_hat).sum()
  union = y.sum() + y_hat.sum() - intersection
  res = -1
  try:
    res = intersection + smooth / union + smooth
    res = res.astype(np.float32)
  except ZeroDivisionError:
    logging.warning('Zero Division Error.')
  except:
    logging.warning('Unexpected Error')
  return res

def dice_coef(y, y_hat):
    y = Flatten()(y)
    y_hat = Flatten()(y_hat)
    intersection = tf.reduce_sum(y*y_hat)
    union = tf.reduce_sum( tf.reduce_sum(y) + tf.reduce_sum(y_hat) + smooth)
    return (2.0*intersection + smooth) / union

def dice_loss(y, y_hat):
    return 1.0 - dice_coef(y, y_hat)
def create_directory(path):
  if not os.path.exists(path):
    os.makedirs(path)

In [5]:
H, W = 512, 512 
np.random.seed(10)
tf.random.set_seed(10)

In [6]:
create_directory('images')

In [9]:
create_directory('masks')

In [11]:
create_directory('results')

In [ ]:
with CustomObjectScope({'iou' : iou, 'dice_coef' : dice_coef,
                        'dice_loss' : dice_loss}):
  model = load_model('/content/drive/MyDrive/Colab Exercises/model.h5')

model.summary()

In [37]:
!wget https://cdn03.ciceksepeti.com/cicek/at4273-1/L/11-li-kirmizi-gul-cicek-buketi-at4273-1-bf4868a0836d4c5898c8f8a1366ed969.jpg -O image3.png

--2022-08-18 07:10:38--  https://cdn03.ciceksepeti.com/cicek/at4273-1/L/11-li-kirmizi-gul-cicek-buketi-at4273-1-bf4868a0836d4c5898c8f8a1366ed969.jpg
Resolving cdn03.ciceksepeti.com (cdn03.ciceksepeti.com)... 23.219.154.141, 23.219.154.133, 2600:1408:c400:68::17db:9a44, ...
Connecting to cdn03.ciceksepeti.com (cdn03.ciceksepeti.com)|23.219.154.141|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [image/jpeg]
Saving to: ‘image3.png’

image3.png              [ <=>                ]  56.01K  --.-KB/s    in 0.004s  

2022-08-18 07:10:39 (13.3 MB/s) - ‘image3.png’ saved [57350]



In [18]:
with CustomObjectScope({'iou' : iou, 'dice_coef' : dice_coef,
                        'dice_loss' : dice_loss}):
    model = load_model('/content/drive/MyDrive/Colab Exercises/model.h5')

def mask(imagePath, model, imHeight, imWidth,
         mask = [255.,255.,255.]):
  create_directory('masks')
  create_directory('results')

  np.random.seed(10)
  tf.random.set_seed(10)

  data = glob(imagePath + '/*')
  for p in tqdm(data, total = len(data)):
    imName = p.split('/')[-1].split('.')[0]
    image = cv2.imread(p, cv2.IMREAD_COLOR)
    h, w, *r = image.shape 
    x = cv2.resize(image, (imWidth, imHeight))
    x = x / 255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis = 0)
  
    y = model.predict(x)[0]
    y = cv2.resize(y, (w,h))
    y = np.expand_dims(y, axis = -1)
    y = y > 0.5

    cv2.imwrite(f'masks/{imName}_mask.png', y*255.0)

    mask = y
    b_mask = np.abs(1-y)

    masked = mask*image
    b_masked = np.concatenate([b_mask,b_mask,b_mask], axis = -1)
    b_masked = b_masked * mask
    f_image = masked + b_masked
    cv2.imwrite(f'results/{imName}_masked.png', f_image)

In [19]:
ip = '/content/images'
mask(ip, model, 512, 512)

  0%|          | 0/2 [00:00<?, ?it/s]

1/1 [==============================] - 4s 4s/step


 50%|█████     | 1/2 [00:03<00:03,  3.64s/it]

1/1 [==============================] - 2s 2s/step


100%|██████████| 2/2 [00:06<00:00,  3.14s/it]


In [15]:
data = glob('images/*')

for p in tqdm(data, total = len(data)):
  imName = p.split('/')[-1].split('.')[0]
  image = cv2.imread(p, cv2.IMREAD_COLOR)
  h, w, *r = image.shape 
  x = cv2.resize(image, (W,H))
  x = x / 255.0
  x = x.astype(np.float32)
  x = np.expand_dims(x, axis = 0)
  
  y = model.predict(x)[0]
  y = cv2.resize(y, (w,h))
  y = np.expand_dims(y, axis = -1)
  #y = y > 0.5

  cv2.imwrite(f'/content/masks/{imName}_mask.png', y*255.0)

  mask = y
  b_mask = np.abs(1-y)

  masked = mask*image
  b_masked = np.concatenate([b_mask,b_mask,b_mask], axis = -1)
  b_masked = b_masked * [255.,255.,255.]
  f_image = masked + b_masked
  cv2.imwrite(f'/content/results/{imName}_masked.png', f_image)

  0%|          | 0/2 [00:00<?, ?it/s]

1/1 [==============================] - 2s 2s/step


 50%|█████     | 1/2 [00:02<00:02,  2.65s/it]

1/1 [==============================] - 2s 2s/step


100%|██████████| 2/2 [00:05<00:00,  2.64s/it]
